# 拼豆图纸生成器 API 使用示例

本示例演示如何使用拼豆图纸生成器的完整API功能，包括:
- 服务状态检查
- 调色板管理
- 图片转换
- 图纸下载
- 自定义调色板使用

确保服务已启动在 http://localhost:3000

## 环境准备

导入必要的库并设置基础配置

In [16]:
import requests
import json
import base64
import os
from datetime import datetime

# API基础URL
BASE_URL = "http://localhost:3000/api"

# 辅助函数
def encode_image_to_base64(image_path):
    """将图片编码为base64"""
    try:
        with open(image_path, 'rb') as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')
    except Exception as e:
        print(f"图片编码失败: {e}")
        return None

def print_response(response, title="响应"):
    """格式化打印响应"""
    print(f"\n--- {title} ---")
    print(f"状态码: {response.status_code}")
    # if response.headers.get('content-type', '').startswith('application/json'):
    #     print(json.dumps(response.json(), indent=2, ensure_ascii=False))
    # else:
    #     print(f"内容类型: {response.headers.get('content-type')}")
    #     print(f"内容大小: {len(response.content)} 字节")

print("环境配置完成")

环境配置完成


## 1. 服务状态检查

首先检查API服务是否正常运行

In [17]:
# 检查服务状态
print("检查服务状态...")
try:
    response = requests.get(f"{BASE_URL}/status", timeout=10)
    print_response(response, "服务状态")

    if response.status_code == 200:
        status_data = response.json()
        print(f"\n服务: {status_data['service']}")
        print(f"状态: {status_data['status']}")
        print(f"运行时间: {status_data['uptime']:.2f} 秒")
        print(f"版本: {status_data['version']}")
        print("服务运行正常")
    else:
        print("服务状态异常")

except requests.exceptions.RequestException as e:
    print(f"连接失败: {e}")
    print("请确保服务已启动在 http://localhost:3000")

检查服务状态...

--- 服务状态 ---
状态码: 200

服务: perler-beads-api
状态: healthy
运行时间: 281.18 秒
版本: 1.0.0
服务运行正常


## 2. 调色板信息获取

获取支持的颜色系统和调色板信息

In [18]:
# 获取基础调色板信息
print("获取调色板信息...")
try:
    response = requests.get(f"{BASE_URL}/palette")
    print_response(response, "调色板信息")

    if response.status_code == 200:
        palette_data = response.json()['data']

        print("\n=== 调色板概览 ===")
        print(f"可用调色板: {palette_data['availablePalettes']}")
        print(f"默认调色板: {palette_data['defaultPalette']}")
        print(f"总颜色数: {palette_data['totalColors']}")
        print(f"支持自定义调色板: {palette_data['supportsCustomPalette']}")

        print("\n=== 颜色系统 ===")
        color_systems = palette_data['colorSystems']
        for i, cs in enumerate(color_systems):
            print(f"{i+1}. {cs['key']}: {cs['name']}")

        # 保存颜色系统供后续使用
        selected_color_system = color_systems[0]['key']
        print(f"\n选择的颜色系统: {selected_color_system}")

except Exception as e:
    print(f"获取调色板信息失败: {e}")

获取调色板信息...

--- 调色板信息 ---
状态码: 200

=== 调色板概览 ===
可用调色板: ['291色', '自定义']
默认调色板: 291色
总颜色数: 291
支持自定义调色板: True

=== 颜色系统 ===
1. MARD: MARD
2. COCO: COCO
3. 漫漫: 漫漫
4. 盼盼: 盼盼
5. 咪小窝: 咪小窝

选择的颜色系统: MARD


## 3. 获取详细颜色信息

获取指定颜色系统的详细颜色信息

In [19]:
# 获取详细的颜色信息
print(f"获取 {selected_color_system} 颜色系统的详细信息...")
try:
    response = requests.get(f"{BASE_URL}/palette?detailed=true&colorSystem={selected_color_system}")
    print_response(response, f"{selected_color_system} 详细颜色信息")

    if response.status_code == 200:
        detailed_data = response.json()['data']
        colors = detailed_data['colors']

        print(f"\n=== {detailed_data['colorSystem']} 颜色系统详情 ===")
        print(f"总颜色数: {detailed_data['totalColors']}")

        print("\n前10个颜色:")
        for i, color in enumerate(colors[:10]):
            print(f"{i+1:2d}. 色号: {color['key']:4s} | 颜色: {color['hex']} | RGB: {color['rgb']}")

        print(f"\n... 共 {len(colors)} 种颜色")

except Exception as e:
    print(f"获取详细颜色信息失败: {e}")

获取 MARD 颜色系统的详细信息...

--- MARD 详细颜色信息 ---
状态码: 200

=== MARD 颜色系统详情 ===
总颜色数: 291

前10个颜色:
 1. 色号: A01  | 颜色: #FAF4C8 | RGB: {'r': 250, 'g': 244, 'b': 200}
 2. 色号: A02  | 颜色: #FFFFD5 | RGB: {'r': 255, 'g': 255, 'b': 213}
 3. 色号: A03  | 颜色: #FEFF8B | RGB: {'r': 254, 'g': 255, 'b': 139}
 4. 色号: A04  | 颜色: #FBED56 | RGB: {'r': 251, 'g': 237, 'b': 86}
 5. 色号: A05  | 颜色: #F4D738 | RGB: {'r': 244, 'g': 215, 'b': 56}
 6. 色号: A06  | 颜色: #FEAC4C | RGB: {'r': 254, 'g': 172, 'b': 76}
 7. 色号: A07  | 颜色: #FE8B4C | RGB: {'r': 254, 'g': 139, 'b': 76}
 8. 色号: A08  | 颜色: #FFDA45 | RGB: {'r': 255, 'g': 218, 'b': 69}
 9. 色号: A09  | 颜色: #FF995B | RGB: {'r': 255, 'g': 153, 'b': 91}
10. 色号: A10  | 颜色: #F77C31 | RGB: {'r': 247, 'g': 124, 'b': 49}

... 共 291 种颜色


## 4. 图片转换（使用默认调色板）

使用默认291色调色板转换图片

In [20]:
# 检查测试图片
test_image = "test_image.png"
if not os.path.exists(test_image):
    print(f"警告: 测试图片 {test_image} 不存在")
    print("请确保在tests目录下有test_image.png文件")
    test_image = None
else:
    print(f"找到测试图片: {test_image}")
    file_size = os.path.getsize(test_image)
    print(f"文件大小: {file_size} 字节")

找到测试图片: test_image.png
文件大小: 164 字节


In [21]:
# 转换图片（如果测试图片存在）
if test_image and os.path.exists(test_image):
    print("开始图片转换...")

    try:
        with open(test_image, 'rb') as f:
            files = {'image': (test_image, f, 'image/png')}
            form_data = {
                'granularity': '15',
                'pixelationMode': 'dominant',
                'selectedPalette': '291色',
                'selectedColorSystem': selected_color_system,
                'similarityThreshold': 10
            }

            response = requests.post(
                f"{BASE_URL}/convert",
                files=files,
                data=form_data,
                timeout=30
            )

        print_response(response, "图片转换结果")

        if response.status_code == 200:
            convert_result = response.json()

            if convert_result['success']:
                data = convert_result['data']

                print("\n=== 转换成功 ===")
                print(f"网格尺寸: {data['gridDimensions']['N']} x {data['gridDimensions']['M']}")
                print(f"总珠子数: {data['totalBeadCount']}")
                print(f"使用颜色数: {len(data['colorCounts'])}")

                print("\n颜色使用统计:")
                for color_hex, info in list(data['colorCounts'].items())[:5]:
                    print(f"  {color_hex}: {info['count']} 个珠子")

                if len(data['colorCounts']) > 5:
                    print(f"  ... 还有 {len(data['colorCounts']) - 5} 种颜色")

                # 保存转换结果供下载使用
                global convert_data
                convert_data = data
                print("\n转换数据已保存，可用于下载图纸")

            else:
                print(f"转换失败: {convert_result}")
        else:
            print(f"转换请求失败: {response.status_code}")

    except Exception as e:
        print(f"转换过程出错: {e}")
else:
    print("跳过图片转换（无测试图片）")

开始图片转换...

--- 图片转换结果 ---
状态码: 200

=== 转换成功 ===
网格尺寸: 15 x 15
总珠子数: 225
使用颜色数: 2

颜色使用统计:
  #FFFFFF: 144 个珠子
  #E7002F: 81 个珠子

转换数据已保存，可用于下载图纸


## 5. 图纸下载

使用转换结果生成并下载图纸文件

In [22]:
# 下载图纸（如果有转换数据）
if 'convert_data' in globals():
    print("生成图纸...")

    try:
        download_data = {
            "pixelData": convert_data['pixelData'],
            "gridDimensions": convert_data['gridDimensions'],
            "colorCounts": convert_data['colorCounts'],
            "totalBeadCount": convert_data['totalBeadCount'],
            "activeBeadPalette": convert_data['activeBeadPalette'],
            "selectedColorSystem": selected_color_system,
            "downloadOptions": {
                "showGrid": True,
                "gridInterval": 10,
                "showCoordinates": True,
                "gridLineColor": "#CCCCCC",
                "includeStats": True,
                "filename": "api_example_pattern"
            }
        }

        response = requests.post(
            f"{BASE_URL}/download",
            json=download_data,
            timeout=30
        )

        print(f"下载请求状态码: {response.status_code}")
        print(f"响应内容类型: {response.headers.get('content-type')}")

        if response.status_code == 200:
            # 保存下载的图片
            output_filename = "api_example_result.png"
            with open(output_filename, 'wb') as f:
                f.write(response.content)

            file_size = len(response.content)
            print(f"\n=== 下载成功 ===")
            print(f"文件大小: {file_size} 字节")
            print(f"保存为: {output_filename}")

            # 验证文件
            if os.path.exists(output_filename) and os.path.getsize(output_filename) > 0:
                print("文件验证成功")
            else:
                print("文件验证失败")

        else:
            print(f"下载失败: {response.status_code}")
            try:
                error_data = response.json()
                print(f"错误信息: {json.dumps(error_data, indent=2, ensure_ascii=False)}")
            except:
                print(f"响应内容: {response.text[:200]}...")

    except Exception as e:
        print(f"下载过程出错: {e}")
else:
    print("跳过图纸下载（无转换数据）")

生成图纸...
下载请求状态码: 200
响应内容类型: image/png

=== 下载成功 ===
文件大小: 43631 字节
保存为: api_example_result.png
文件验证成功


## 6. 自定义调色板验证

演示如何验证自定义调色板

In [23]:
# 创建自定义调色板
print("创建自定义调色板...")

# 新格式 (v3.0) - 推荐使用
custom_palette_v3 = {
    "version": "3.0",
    "selectedHexValues": ["#E7002F", "#FEFFFF", "#00FF00", "#0000FF", "#FFFF00"],
    "exportDate": datetime.now().isoformat(),
    "totalColors": 5
}

print("自定义调色板 (v3.0):")
print(json.dumps(custom_palette_v3, indent=2, ensure_ascii=False))

创建自定义调色板...
自定义调色板 (v3.0):
{
  "version": "3.0",
  "selectedHexValues": [
    "#E7002F",
    "#FEFFFF",
    "#00FF00",
    "#0000FF",
    "#FFFF00"
  ],
  "exportDate": "2025-06-03T12:01:09.705779",
  "totalColors": 5
}


In [24]:
# 验证自定义调色板
print("\n验证自定义调色板...")

try:
    response = requests.post(
        f"{BASE_URL}/palette",
        json={"customPalette": custom_palette_v3},
        headers={"Content-Type": "application/json"}
    )

    print_response(response, "调色板验证结果")

    if response.status_code == 200:
        result = response.json()
        if result['success']:
            data = result['data']
            print(f"\n=== 验证成功 ===")
            print(f"总颜色数: {data['totalColors']}")
            print(f"版本: {data['version']}")
            print(f"消息: {data['message']}")

            print("\n验证的颜色:")
            for color in data['validatedColors']:
                rgb = color['rgb']
                print(f"  {color['key']}: {color['hex']} (RGB: {rgb['r']}, {rgb['g']}, {rgb['b']})")

            # 保存验证后的调色板供后续使用
            global validated_custom_palette
            validated_custom_palette = custom_palette_v3
            print("\n自定义调色板验证成功，可用于图片转换")
        else:
            print(f"验证失败: {result['error']}")
    else:
        print(f"验证请求失败: {response.status_code}")

except Exception as e:
    print(f"验证过程出错: {e}")


验证自定义调色板...

--- 调色板验证结果 ---
状态码: 200

=== 验证成功 ===
总颜色数: 5
版本: 3.0
消息: 自定义调色板验证成功

验证的颜色:
  #E7002F: #E7002F (RGB: 231, 0, 47)
  #FEFFFF: #FEFFFF (RGB: 254, 255, 255)
  #00FF00: #00FF00 (RGB: 0, 255, 0)
  #0000FF: #0000FF (RGB: 0, 0, 255)
  #FFFF00: #FFFF00 (RGB: 255, 255, 0)

自定义调色板验证成功，可用于图片转换


## 7. 使用自定义调色板转换图片

演示如何使用验证过的自定义调色板进行图片转换

In [25]:
# 使用自定义调色板转换图片
if test_image and os.path.exists(test_image) and 'validated_custom_palette' in globals():
    print("使用自定义调色板转换图片...")

    try:
        with open(test_image, 'rb') as f:
            files = {'image': (test_image, f, 'image/png')}
            form_data = {
                'granularity': '25',
                'pixelationMode': 'average',
                'selectedPalette': '自定义',
                'customPalette': json.dumps(validated_custom_palette)
            }

            response = requests.post(
                f"{BASE_URL}/convert",
                files=files,
                data=form_data,
                timeout=30
            )

        print_response(response, "自定义调色板转换结果")

        if response.status_code == 200:
            convert_result = response.json()

            if convert_result['success']:
                data = convert_result['data']

                print("\n=== 自定义调色板转换成功 ===")
                print(f"网格尺寸: {data['gridDimensions']['N']} x {data['gridDimensions']['M']}")
                print(f"总珠子数: {data['totalBeadCount']}")
                print(f"使用颜色数: {len(data['colorCounts'])}")
                print(f"调色板来源: {data['processingParams']['paletteSource']}")

                print("\n自定义调色板颜色使用:")
                for color_hex, info in data['colorCounts'].items():
                    print(f"  {color_hex}: {info['count']} 个珠子")

                # 保存自定义调色板转换结果
                global custom_convert_data
                custom_convert_data = data
                print("\n自定义调色板转换数据已保存")

            else:
                print(f"转换失败: {convert_result}")
        else:
            print(f"转换请求失败: {response.status_code}")

    except Exception as e:
        print(f"自定义调色板转换出错: {e}")
else:
    print("跳过自定义调色板转换（缺少必要条件）")

使用自定义调色板转换图片...

--- 自定义调色板转换结果 ---
状态码: 200

=== 自定义调色板转换成功 ===
网格尺寸: 25 x 25
总珠子数: 625
使用颜色数: 2
调色板来源: custom

自定义调色板颜色使用:
  #FEFFFF: 370 个珠子
  #E7002F: 255 个珠子

自定义调色板转换数据已保存


## 8. 下载自定义调色板图纸

生成并下载使用自定义调色板的图纸

In [26]:
# 下载自定义调色板图纸
if 'custom_convert_data' in globals():
    print("生成自定义调色板图纸...")

    try:
        download_data = {
            "pixelData": custom_convert_data['pixelData'],
            "gridDimensions": custom_convert_data['gridDimensions'],
            "colorCounts": custom_convert_data['colorCounts'],
            "totalBeadCount": custom_convert_data['totalBeadCount'],
            "activeBeadPalette": custom_convert_data['activeBeadPalette'],
            "selectedColorSystem": custom_convert_data.get('selectedColorSystem', 'MARD'),  # 添加这个字段
            "downloadOptions": {
                "showGrid": True,
                "gridInterval": 5,
                "showCoordinates": True,
                "gridLineColor": "#999999",
                "includeStats": True,
                "filename": "custom_palette_pattern"
            }
        }

        # 添加调试信息
        print("发送的数据结构:")
        print(f"- pixelData: {len(custom_convert_data['pixelData'])} 行")
        print(f"- gridDimensions: {custom_convert_data['gridDimensions']}")
        print(f"- colorCounts: {len(custom_convert_data['colorCounts'])} 种颜色")
        print(f"- totalBeadCount: {custom_convert_data['totalBeadCount']}")
        print(f"- activeBeadPalette: {type(custom_convert_data['activeBeadPalette'])}")

        response = requests.post(
            f"{BASE_URL}/download",
            json=download_data,
            timeout=30
        )

        print(f"下载请求状态码: {response.status_code}")
        print(f"响应内容类型: {response.headers.get('content-type')}")

        if response.status_code == 200:
            # 保存下载的图片
            output_filename = "custom_palette_result.png"
            with open(output_filename, 'wb') as f:
                f.write(response.content)

            file_size = len(response.content)
            print(f"\n=== 自定义调色板图纸下载成功 ===")
            print(f"文件大小: {file_size} 字节")
            print(f"保存为: {output_filename}")
            print(f"使用了 {len(custom_convert_data['colorCounts'])} 种自定义颜色")

        else:
            print(f"下载失败: {response.status_code}")
            # 添加详细的错误信息
            try:
                error_data = response.json()
                print(f"错误详情: {json.dumps(error_data, indent=2, ensure_ascii=False)}")
            except:
                print(f"响应内容: {response.text[:200]}...")

    except Exception as e:
        print(f"自定义调色板图纸下载出错: {e}")
else:
    print("跳过自定义调色板图纸下载（无转换数据）")

生成自定义调色板图纸...
发送的数据结构:
- pixelData: 25 行
- gridDimensions: {'N': 25, 'M': 25, 'width': 25, 'height': 25}
- colorCounts: 2 种颜色
- totalBeadCount: 625
- activeBeadPalette: <class 'list'>
下载请求状态码: 200
响应内容类型: image/png

=== 自定义调色板图纸下载成功 ===
文件大小: 77205 字节
保存为: custom_palette_result.png
使用了 2 种自定义颜色


## 9. API功能总结

展示本示例中演示的所有功能

In [27]:
# 功能总结
print("=== API 功能演示总结 ===")
print("\n已演示的功能:")
print("1. 服务状态检查 - GET /api/status")
print("2. 获取调色板信息 - GET /api/palette")
print("3. 获取详细颜色信息 - GET /api/palette?detailed=true&colorSystem=MARD")
print("4. 图片转换（默认调色板）- POST /api/convert")
print("5. 图纸下载 - POST /api/download")
print("6. 自定义调色板验证 - POST /api/palette")
print("7. 图片转换（自定义调色板）- POST /api/convert")
print("8. 自定义调色板图纸下载 - POST /api/download")

print("\n生成的文件:")
generated_files = []
for filename in ["api_example_result.png", "custom_palette_result.png"]:
    if os.path.exists(filename):
        size = os.path.getsize(filename)
        generated_files.append(f"  {filename} ({size} 字节)")

if generated_files:
    for file_info in generated_files:
        print(file_info)
else:
    print("  无文件生成（可能缺少测试图片）")

print("\n支持的功能特性:")
print("- 291色完整调色板")
print("- 5种色号系统 (MARD, COCO, 漫漫, 盼盼, 咪小窝)")
print("- 自定义调色板 (v3.0格式)")
print("- 向后兼容旧格式")
print("- 多种像素化模式")
print("- 可调节的颜色相似度合并")
print("- 高质量图纸生成")
print("- 网格线和坐标系")
print("- 颜色使用统计")

print("\n演示完成！")

=== API 功能演示总结 ===

已演示的功能:
1. 服务状态检查 - GET /api/status
2. 获取调色板信息 - GET /api/palette
3. 获取详细颜色信息 - GET /api/palette?detailed=true&colorSystem=MARD
4. 图片转换（默认调色板）- POST /api/convert
5. 图纸下载 - POST /api/download
6. 自定义调色板验证 - POST /api/palette
7. 图片转换（自定义调色板）- POST /api/convert
8. 自定义调色板图纸下载 - POST /api/download

生成的文件:
  api_example_result.png (43631 字节)
  custom_palette_result.png (77205 字节)

支持的功能特性:
- 291色完整调色板
- 5种色号系统 (MARD, COCO, 漫漫, 盼盼, 咪小窝)
- 自定义调色板 (v3.0格式)
- 向后兼容旧格式
- 多种像素化模式
- 可调节的颜色相似度合并
- 高质量图纸生成
- 网格线和坐标系
- 颜色使用统计

演示完成！


## 附录: 常用配置参数

以下是一些常用的配置参数组合，可根据需要调整：

In [27]:
# 常用配置参数参考
print("=== 常用配置参数参考 ===")

configurations = {
    "小尺寸快速预览": {
        "granularity": "15",
        "pixelationMode": "dominant",
        "similarityThreshold": "40"
    },
    "中等尺寸平衡": {
        "granularity": "30",
        "pixelationMode": "average",
        "similarityThreshold": "30"
    },
    "高精度大图": {
        "granularity": "60",
        "pixelationMode": "average",
        "similarityThreshold": "20"
    },
    "简化颜色": {
        "granularity": "25",
        "pixelationMode": "dominant",
        "similarityThreshold": "50"
    }
}

for name, config in configurations.items():
    print(f"\n{name}:")
    for key, value in config.items():
        print(f"  {key}: {value}")

print("\n下载选项参考:")
download_options = {
    "基础图纸": {
        "cellSize": 25,
        "showGrid": True,
        "gridInterval": 10,
        "showCoordinates": False,
        "includeStats": True
    },
    "精细图纸": {
        "cellSize": 30,
        "showGrid": True,
        "gridInterval": 5,
        "showCoordinates": True,
        "includeStats": True
    },
    "打印用图纸": {
        "cellSize": 35,
        "showGrid": True,
        "gridInterval": 10,
        "showCoordinates": True,
        "gridLineColor": "#000000",
        "includeStats": True
    }
}

for name, options in download_options.items():
    print(f"\n{name}:")
    for key, value in options.items():
        print(f"  {key}: {value}")

=== 常用配置参数参考 ===

小尺寸快速预览:
  granularity: 15
  pixelationMode: dominant
  similarityThreshold: 40

中等尺寸平衡:
  granularity: 30
  pixelationMode: average
  similarityThreshold: 30

高精度大图:
  granularity: 60
  pixelationMode: average
  similarityThreshold: 20

简化颜色:
  granularity: 25
  pixelationMode: dominant
  similarityThreshold: 50

下载选项参考:

基础图纸:
  cellSize: 25
  showGrid: True
  gridInterval: 10
  showCoordinates: False
  includeStats: True

精细图纸:
  cellSize: 30
  showGrid: True
  gridInterval: 5
  showCoordinates: True
  includeStats: True

打印用图纸:
  cellSize: 35
  showGrid: True
  gridInterval: 10
  showCoordinates: True
  gridLineColor: #000000
  includeStats: True
